In [ ]:
# !pip install -U llama-index-readers-file

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "5"

In [2]:
import torch
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex, Document, SimpleKeywordTableIndex
from llama_index.llms.huggingface import HuggingFaceLLM
from huggingface_hub import hf_hub_download
from llama_index.core import Settings
from llama_index.core import StorageContext
from llama_index.core import PromptTemplate

ModuleNotFoundError: No module named 'torch'

In [4]:
# Step 1: Set up the HuggingFace LLM
llm = HuggingFaceLLM(
    model_name="aaditya/OpenBioLLM-Llama3-8B",
    tokenizer_name="aaditya/OpenBioLLM-Llama3-8B",
    model_kwargs={"torch_dtype": torch.bfloat16},
    generate_kwargs={"temperature": 0.7, "top_k": 50, "top_p": 0.95},
    device_map="cuda",
)

Settings.llm=llm
Settings.embed_model="local:BAAI/bge-small-en-v1.5" 

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]


RuntimeError: No CUDA GPUs are available

In [4]:
# Step 2: Load the document (assuming it's stored in a text file)
document_path = "/raid/ganesh/vishak/ashutosh/temp/temp1"  # Path to your document
reader = SimpleDirectoryReader(input_dir=document_path)
documents = reader.load_data()

nodes = Settings.node_parser.get_nodes_from_documents(documents)

# initialize storage context (by default it's in-memory)
storage_context = StorageContext.from_defaults()
storage_context.docstore.add_documents(nodes)

vector_index = VectorStoreIndex(nodes, storage_context=storage_context)
keyword_index = SimpleKeywordTableIndex(nodes, storage_context=storage_context)

retriever = vector_index.as_retriever()

In [5]:
def generate_response(retrieved_nodes, query_str, qa_prompt, llm):
    context_str = "\n\n".join([r.get_content() for r in retrieved_nodes])
    fmt_qa_prompt = qa_prompt.format(
        context_str=context_str, query_str=query_str
    )
    response = llm.complete(fmt_qa_prompt)
    return str(response), fmt_qa_prompt

In [9]:
Question = "Was the patient considered for curative intent treatment?"

retrieved_nodes = retriever.retrieve(Question)


qa_prompt = PromptTemplate(
    """\
    Context information is below.
    I want you to act like a breast cancer expert/doctor.
    I will give you summary of a breast cancer patient's stay in the hospital, you will evaluate it and answer a set of questions as 'Yes' or 'No' only.
    ---------------------
    {context_str}
    ---------------------
    Given the context information and not prior knowledge, answer the query.
    Query: {query_str}
    Answer: \
    """
)
response_text= generate_response(retrieved_nodes, Question, qa_prompt, llm )
print("\n\n\n", response_text[0])

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.





  Yes


In [37]:
questions = """
    Was the patient considered for curative intent treatment?
    Did the patient undergo a Mammogram (MMG)?
    Did the patient undergo a Ultrasound (USG)?
    Did the patient undergo a physical examination of both breasts?
    Did the patient undergo a physical examination of axilla?
    Is the patient undergoing upfront surgery?
    Does the patient have a pre-operatively confirmed histological or cytological diagnosis?
    Was the patient considered for curative intent treatment?
    Did the patient undergo image-guided axillary staging?
    Was the patient considered for curative intent treatment?
    Does the biopsy report of the patient include Prognostic factors?
    Does the biopsy report of the patient include predictive factors?
    Is the breast cancer grade recorded in the biopsy report of the patient?
    Is ER (Estrogen Receptor status) recorded in the biopsy report of the patient?
    Is PR (Progesterone Receptor status) recorded in the biopsy report of the patient?
    Is HER2Neu status recorded in the biopsy report of the patient?
    Is the patient undergoing upfront systemic therapy?
    Did the patient have a core biopsy?
    Is the patient undergoing upfront systemic therapy?
    Does the patient have ER (Estrogen Receptor status) recorded?
    Does the patient have PR (Progesterone Receptor status) recorded?
    Does the patient have HER2Neu status recorded?
    Was the patient considered for curative intent treatment?
    What is the time interval from the date of the first diagnostic examination to the date of surgery?
    What is the time interval from the date of the first diagnostic examination to the start of systemic therapy?
    Is the time interval from the date of the first diagnostic examination to the date of surgery/start of systemic therapy less than 6 weeks?
    Was the decision to consider the patient for curative intent treatment taken by a multidisciplinary team (Tumor board)?
    Is the patient diagnosed with stage I breast cancer?
    Is the patient diagnosed with primary operable stage II breast cancer?
    Did the patient not undergo a baseline USG (Ultrasound) as a staging test?
    Did the patient not undergo a baseline bone scan as a staging test?
    Is the patient diagnosed with stage III breast cancer?
    Did the patient undergo a baseline USG of the abdomen (USG abd)?
    Did the patient undergo a baseline Chest X-Ray (CXR)?
    Did the patient undergo baseline Liver Function Tests (LFT)?
    Did the patient undergo a baseline bone scan?
    Alternatively, did the patient undergo a PET Scan?
    Alternatively, did the patient undergo a CT Thorax with abdomen?
    Was the patient planned for Breast Conserving Surgery (BCS)?
    Did the patient receive tumor localization before pre-operative systemic therapy?
    Is the patient at high risk for breast cancer?
    Does the patient have triple negative breast cancer?
    Did the patient receive genetic counselling?
    Did the patient get genetic testing before starting adjuvant therapy with olaparib?
    Does the patient have HER2 Neu scored as 2+?
    Was the HER2 Neu 2+ status confirmed by FISH (Fluorescence In Situ Hybridization)?
    Did the breast cancer patient have Pre-Anesthetic Checkup (PAC) documented prior to surgery?
    Did the patient with breast cancer have a mammogram at the time of diagnosis?
    Was the mammogram report of the patient documented using the BIRADS (Breast Imaging-Reporting and Data System) standard?
    Is the patient receiving antiHer2 therapy?
    Is the antiHer2 therapy being used as neoadjuvant therapy?
    Is the antiHer2 therapy being used as adjuvant therapy?
    Is the patient receiving at least 12 weeks of antiHer2 therapy?
    Is the patient node positive for breast cancer?
    Was the node-positive patient advised to undergo chemotherapy?
    Is the patient hormone receptor (ER/PR) positive?
    Did the hormone receptor (ER/PR) positive patient receive tamoxifen treatment?
    Did the hormone receptor (ER/PR) positive patient receive aromatase inhibitor treatment?
    Has the patient started chemotherapy?
    Prior to starting chemotherapy, was the Complete Blood Count (CBC) documented?
    Prior to starting chemotherapy, was the Renal Function Test (RFT) documented?
    Prior to starting chemotherapy, was the Liver Function Test (LFT) documented?
    Prior to starting chemotherapy, was the Random Blood Sugar (RBS) documented?
    Has the patient started anthracycline and/or trastuzumab-based chemotherapy?
    Prior to starting anthracycline-based chemotherapy, was an ECHO or MUGA scan report documented?
    Prior to starting trastuzumab-based chemotherapy, was an ECHO or MUGA scan report documented?
    During neoadjuvant chemotherapy, was there documented reassessment of the lump status at each visit?
    Did the patient receiving chemotherapy have documented toxicities to the chemotherapy?
    Was the patient diagnosed with stage I invasive breast cancer and undergone a single operation?
    Was the patient diagnosed with stage II invasive breast cancer and undergone a single operation?
    Was the patient diagnosed with stage III invasive breast cancer and undergone a single operation?
    Was the patient diagnosed with in-situ breast cancer and undergone a single surgery?
    Was the patient diagnosed with stage I breast cancer?
    Alternatively, was the patient diagnosed with stage II breast cancer?
    Was the patient's axilla clinically assessed as negative?
    Did the patient receive any pre-surgery therapy (PST)?
    If the patient received no PST, did they undergo a Sentinel Lymph Node Biopsy (SLNB)?
    Did the patient undergo Sentinel Lymph Node Biopsy (SLNB)?
    Were less than 5 nodes removed during the SLNB?
    Was the patient diagnosed with Ductal Carcinoma In Situ (DCIS)?
    Did the patient undergoing surgery for DCIS not undergo axillary clearance?
    Was the patient diagnosed with invasive/infiltrating breast cancer?
    Did the patient undergo breast conservation surgery?
    Did the patient receive adjuvant radiotherapy after breast conservation surgery?
    Does the patient have involvement of axillary lymph nodes (≥ pN2a)?
    Did the patient undergo a mastectomy?
    Did the patient receive post-mastectomy radiation therapy to the chest wall and supraclavicular fossa (SCF)?
    Was the patient diagnosed with high-grade breast cancer?
    Does the patient have focal positive margins?
    Is the patient aged 50 years or younger?
    Did the patient receive a lumpectomy?
    Did the patient receive a lumpectomy cavity boost?
    Was the internal mammary node diagnosed with cancer in the patient?
    Did the patient receive radiotherapy to the internal mammary node?
    Was the patient clinically diagnosed as node positive?
    Did the patient undergo neoadjuvant chemotherapy?
    Did the patient receive adjuvant radiotherapy?
    Is the patient being treated with curative intent?
    Did the treatment start within 6 weeks of diagnosis?
    Did the patient start radiation therapy (RT) within 7 months after surgery?
    Did the patient start chemotherapy within 12 weeks post-surgery?
    Was the breast cancer patient followed up within 10 to 12 weeks after completing treatment?
    Did the patient start systemic (adjuvant/neoadjuvant) treatment?
    Did the patient complete systemic therapy within 8 months of starting?
    Note: This excludes patients on anti-HER2 and hormonal therapy.
"""
questions = [ques.strip() for ques in questions.strip().split("\n")]
len(questions)

105

In [38]:
questions

['Was the patient considered for curative intent treatment?',
 'Did the patient undergo a Mammogram (MMG)?',
 'Did the patient undergo a Ultrasound (USG)?',
 'Did the patient undergo a physical examination of both breasts?',
 'Did the patient undergo a physical examination of axilla?',
 'Is the patient undergoing upfront surgery?',
 'Does the patient have a pre-operatively confirmed histological or cytological diagnosis?',
 'Was the patient considered for curative intent treatment?',
 'Did the patient undergo image-guided axillary staging?',
 'Was the patient considered for curative intent treatment?',
 'Does the biopsy report of the patient include Prognostic factors?',
 'Does the biopsy report of the patient include predictive factors?',
 'Is the breast cancer grade recorded in the biopsy report of the patient?',
 'Is ER (Estrogen Receptor status) recorded in the biopsy report of the patient?',
 'Is PR (Progesterone Receptor status) recorded in the biopsy report of the patient?',
 'I

In [40]:
from tqdm import tqdm

outputs=  []
for i in tqdm(range(len(questions)), "Processing Query"):
    query = questions[i]
    retrieved_nodes = retriever.retrieve(query)
    qa_prompt = PromptTemplate(
        """\
        Context information is below.
        I want you to act like a breast cancer expert/doctor.
        I will give you summary of a breast cancer patient's stay in the hospital, you will evaluate it and answer a set of questions as 'Yes' or 'No' only.
        ---------------------
        {context_str}
        ---------------------
        Given the context information and not prior knowledge, answer the query.
        Query: {query_str}
        Answer: \
        """
    )
    response_text= generate_response(retrieved_nodes, query, qa_prompt, llm)
    outputs.append(response_text)
    with open("output.txt", "a") as f:
        f.write(response_text[0]+"\n")


with open("log.txt", "a") as f:
    for i in range(len(outputs)):
        f.write(outputs[i][0] + "\n" + outputs[i][1]+ "\n")


Processing Query: 100%|██████████| 105/105 [00:41<00:00,  2.52it/s]


## Using Lora finetuned model


In [9]:
!pip install -U bitsandbytes

In [3]:
import torch 
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

base_model_id = "mistralai/Mistral-7B-v0.1" 
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True, bnb_4bit_use_double_quant=True, 
    bnb_4bit_quant_type="nf4", 
    bnb_4bit_compute_dtype=torch.bfloat16 
    )

base_model = AutoModelForCausalLM.from_pretrained(
    base_model_id, # Mistral, same as before 
    quantization_config=bnb_config,
    device_map="auto", 
    trust_remote_code=True)

eval_tokenizer = AutoTokenizer.from_pretrained(base_model_id, add_bos_token=True, trust_remote_code=True)

from peft import PeftModel

ft_model = PeftModel.from_pretrained(base_model, "ZEECO1/CancerLLM-Mistral7b/checkpoint-500")

eval_prompt = " what are the drugs against lung cancer: # " 
model_input = eval_tokenizer(eval_prompt, return_tensors="pt").to("cuda")

ft_model.eval() 
with torch.no_grad(): 
    print(eval_tokenizer.decode(ft_model.generate(**model_input, max_new_tokens=100, repetition_penalty=1.15)[0], skip_special_tokens=True))

The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
CUDA is required but not available for bitsandbytes. Please consider installing the multi-platform enabled version of bitsandbytes, which is currently a work in progress. Please check currently supported platforms and installation instructions at https://huggingface.co/docs/bitsandbytes/main/en/installation#multi-backend


RuntimeError: CUDA is required but not available for bitsandbytes. Please consider installing the multi-platform enabled version of bitsandbytes, which is currently a work in progress. Please check currently supported platforms and installation instructions at https://huggingface.co/docs/bitsandbytes/main/en/installation#multi-backend